# Batching & Feature extraction
Author: P.C.O. Maseland <br>
Date: 03 December 2021 <br>

This ipynb file contains script which uses the combined dataset, calculates batches with sliding window and uses feature extraction to add rolling slope, rolling mean, and rolling sd features.

First we retrieve original time series, then we split these in batches and in these batches the feature extraction takes place.

In [1]:
#Import packages/libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

%matplotlib inline

In [2]:
path ="Merged_acc_gyr_mag.csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,index_acc,id,magnitude_chest_acc,magnitude_forearm_acc,magnitude_head_acc,magnitude_shin_acc,magnitude_thigh_acc,magnitude_upperarm_acc,magnitude_waist_acc,...,magnitude_waist_gyr,magnitude_chest_mag,magnitude_forearm_mag,magnitude_head_mag,magnitude_shin_mag,magnitude_thigh_mag,magnitude_upperarm_mag,magnitude_waist_mag,Subject ID,Activity
0,0,0,1,9.677577,9.851141,9.808246,10.060116,9.608746,10.188644,9.945071,...,0.038780,51.233096,46.140829,49.391103,38.61178,43.591538,47.442423,54.303116,10,0
1,1,1,2,9.553391,9.864483,9.771217,10.050983,9.600218,10.205661,9.913557,...,0.040448,51.103010,46.381310,49.305273,38.61178,43.591538,47.442423,54.303116,10,0
2,2,2,3,9.551121,9.864997,9.795905,10.057417,9.553888,10.198437,9.966304,...,0.034702,51.103010,46.381310,49.345280,38.61178,43.591538,47.442423,54.303116,10,0
3,3,3,4,9.596779,9.878026,9.762490,10.057711,9.562705,10.190598,10.009170,...,0.025838,51.103010,46.140690,49.345280,38.61178,43.591538,47.442423,54.303116,10,0
4,4,4,5,9.619934,9.851603,9.821960,10.067420,9.604711,10.197248,9.931238,...,0.016002,51.103010,45.682645,49.345280,38.61178,43.591538,47.442423,54.303116,10,0


### Working with csv Tempfile

In [3]:
#get column names
df.columns

Index(['Unnamed: 0', 'index_acc', 'id', 'magnitude_chest_acc',
       'magnitude_forearm_acc', 'magnitude_head_acc', 'magnitude_shin_acc',
       'magnitude_thigh_acc', 'magnitude_upperarm_acc', 'magnitude_waist_acc',
       'magnitude_chest_gyr', 'magnitude_forearm_gyr', 'magnitude_head_gyr',
       'magnitude_shin_gyr', 'magnitude_thigh_gyr', 'magnitude_upperarm_gyr',
       'magnitude_waist_gyr', 'magnitude_chest_mag', 'magnitude_forearm_mag',
       'magnitude_head_mag', 'magnitude_shin_mag', 'magnitude_thigh_mag',
       'magnitude_upperarm_mag', 'magnitude_waist_mag', 'Subject ID',
       'Activity'],
      dtype='object')

In [4]:
#drop columns
df = df.drop(columns=['Unnamed: 0'])

In [6]:
df.shape

(3179067, 25)

In [ ]:
#Distribution of classes
distlabels = Counter(df['Activity'])
distlabels

Counter({0: 320066,
         1: 362705,
         2: 68259,
         3: 479423,
         4: 513768,
         5: 478761,
         6: 473147,
         7: 482938})

## 1. Make Copy to do moddeling on
For adjusting the dataset only using output='Activity' and feature columns (magnitudes)

In [8]:
df_try = df
df_try.columns

Index(['index_acc', 'id', 'magnitude_chest_acc', 'magnitude_forearm_acc',
       'magnitude_head_acc', 'magnitude_shin_acc', 'magnitude_thigh_acc',
       'magnitude_upperarm_acc', 'magnitude_waist_acc', 'magnitude_chest_gyr',
       'magnitude_forearm_gyr', 'magnitude_head_gyr', 'magnitude_shin_gyr',
       'magnitude_thigh_gyr', 'magnitude_upperarm_gyr', 'magnitude_waist_gyr',
       'magnitude_chest_mag', 'magnitude_forearm_mag', 'magnitude_head_mag',
       'magnitude_shin_mag', 'magnitude_thigh_mag', 'magnitude_upperarm_mag',
       'magnitude_waist_mag', 'Subject ID', 'Activity'],
      dtype='object')

In [9]:
#only maintain features and output columns
df_try = df_try.drop(columns=['index_acc', 'id'])
df_try.head(1)

,magnitude_chest_acc,magnitude_forearm_acc,magnitude_head_acc,magnitude_shin_acc,magnitude_thigh_acc,magnitude_upperarm_acc,magnitude_waist_acc,magnitude_chest_gyr,magnitude_forearm_gyr,magnitude_head_gyr,...,magnitude_waist_gyr,magnitude_chest_mag,magnitude_forearm_mag,magnitude_head_mag,magnitude_shin_mag,magnitude_thigh_mag,magnitude_upperarm_mag,magnitude_waist_mag,Subject ID,Activity
0,9.677577,9.851141,9.808246,10.060116,9.608746,10.188644,9.945071,0.061382,0.013997,0.215952,...,0.03878,51.233096,46.140829,49.391103,38.61178,43.591538,47.442423,54.303116,10,0


In [ ]:
df_try.shape

(3179067, 23)

### 1.1 Make time serie batches

#### 1.1.1 Unique combinations list
- list of all label combinations from which the batches will be made. 
- A batch can only be of one label combination (subject/activity)

In [10]:
def uniqueCombos(dataframe):
    uniquecombinations = dataframe.groupby(['Activity', 'Subject ID']).size().reset_index().rename(columns={0:'count'})
    uniquecombinations = uniquecombinations.drop(['count'], axis=1)
    
    uniquecombinations = uniquecombinations.values.tolist()
    
    return uniquecombinations

In [11]:
uniquecombinations = uniqueCombos(df_try)

In [12]:
len(uniquecombinations)

118

##### 1.1.2.-1 Go through sample and create timeseries
- Make timeserie samples by using the uniquecombinations of subjectID and Activity
- add stride

In [13]:
def create_timeserie(activity, subject_id, dataframe):   
    temp_ts = dataframe.loc[(dataframe['Activity'] == activity) & (dataframe['Subject ID'] == subject_id)]
    
    return temp_ts

##### 1.1.2.-2 Go through timeserie and remove begin and end part
These parts turn out to be not representable for the patterns. This is about 200 ms both ends.

In [15]:
def remove_beg_end(timeserie):
    timeserie = timeserie.drop(timeserie.index[0:200])
    timeserie = timeserie.drop(timeserie.index[-200:])
    
    return timeserie

##### 1.1.3 Create batches function
Go through sample and create batches
- use coordinationlist: Startpoint and endpoint to create batches
- save them in a new list 

Create batches with 1000ms with 50% overlap (500ms)

In [16]:
def create_batchlist(dataframe, batchsize, stride):
    batchlist = []  
    
    #get all unique combination pairs
    uniquecombinations = uniqueCombos(dataframe)
    
    for pair in uniquecombinations:
        #get combo
        activity = pair[0]
        subject_id = pair[1]
        
        #get timeserie of combo
        timeserie = create_timeserie(activity, subject_id, dataframe)
        timeserie = remove_beg_end(timeserie)
        
        start_coo = 0
        end_coo = batchsize
        
        #create batches of timeserie
        while end_coo < len(timeserie):
            #print('start: '+ str(start_coo) + ' end: ' + str(end_coo))
            batch = timeserie[start_coo:end_coo]
            batchlist.append(batch)
            
            start_coo += stride
            end_coo += stride
        
        print('Timeserie (' + str(activity) + ' ' + str(subject_id) + ') is batched')
    
    return batchlist

In [17]:
#create batches of 1000ms with 500ms overlap
batchlist = create_batchlist(df_try, 1000, 500)

Timeserie (0 1) is batched
Timeserie (0 2) is batched
Timeserie (0 3) is batched
Timeserie (0 4) is batched
Timeserie (0 5) is batched
Timeserie (0 6) is batched
Timeserie (0 7) is batched
Timeserie (0 8) is batched
Timeserie (0 9) is batched
Timeserie (0 10) is batched
Timeserie (0 11) is batched
Timeserie (0 12) is batched
Timeserie (0 13) is batched
Timeserie (0 14) is batched
Timeserie (0 15) is batched
Timeserie (1 1) is batched
Timeserie (1 3) is batched
Timeserie (1 4) is batched
Timeserie (1 5) is batched
Timeserie (1 6) is batched
Timeserie (1 7) is batched
Timeserie (1 8) is batched
Timeserie (1 9) is batched
Timeserie (1 10) is batched
Timeserie (1 11) is batched
Timeserie (1 12) is batched
Timeserie (1 13) is batched
Timeserie (1 14) is batched
Timeserie (1 15) is batched
Timeserie (2 1) is batched
Timeserie (2 2) is batched
Timeserie (2 3) is batched
Timeserie (2 4) is batched
Timeserie (2 5) is batched
Timeserie (2 7) is batched
Timeserie (2 8) is batched
Timeserie (2 9) 

In [18]:
batchlist[2].shape

(1000, 23)

In [19]:
#nr of total batches
len(batchlist)

6084

In [20]:
# see the shape of the toal batchlist
arr = np.array(batchlist)
arr.shape

(6084, 1000, 23)

### 1.2 Add statistical derivatives features
Calculate statistical derivatives inside each batch:
- Slopes
- Rolling mean
- Rolling SD

With 5ms split used for rolling features

In [21]:
def add_slopes(batchlist, stepsize):
    new_batchlist = []
    count = 0
    
    #calc slope between points
    for batch in batchlist:
        #for each magnitude look at the next row and calc slope
        mag_cols = [col for col in batch.columns if 'magnitude' in col]
        for col in mag_cols:
            slopecol = []

            #name new col
            col_name = col + '_slope'

            #calc slope with stepsize for each row
            temp_valuelist =  list(batch[col])
            for row in range(0, len(temp_valuelist)): 
                if row < (len(temp_valuelist)-stepsize):
                    delta_y = temp_valuelist[row]-temp_valuelist[row+stepsize]
                    delta_x = stepsize
                    slope = np.around((delta_y/delta_x), 5)
                    slopecol.append(slope)
                else: 
                    delta_y = temp_valuelist[row]-temp_valuelist[row-stepsize]
                    delta_x = stepsize
                    slope = np.around((delta_y/delta_x), 5)
                    slopecol.append(slope)
            batch[col_name] = slopecol 
            
        new_batchlist.append(batch)
        print('Batch: ' + str(count) + ' Complete!')
        count += 1
        
    return new_batchlist

In [22]:
def rolling_stats(batchlist, mean=False, SD=False):
    new_batchlist = []
    count = 0
    
    #calc slope between points
    for batch in batchlist:
        #for each magnitude look at the next row and calc slope
        mag_cols = [col for col in batch.columns if 'magnitude' in col and 'slope' not in col]
        for col in mag_cols:

            #name new col
            col_name_mean = col + '_mean'
            col_name_sd = col + '_sd' 

            #calc mean with rolling
            temp_valuelist =  batch[col]
            if mean == True:
              temp_valuelist_mean = temp_valuelist.rolling(5).mean()
              batch[col_name_mean] = temp_valuelist_mean

            if SD == True:
              temp_valuelist_sd = temp_valuelist.rolling(5).std()
              batch[col_name_sd] = temp_valuelist_sd
            
        new_batchlist.append(batch)
        print('Batch: ' + str(count) + ' Complete!')
        count += 1
        
    return new_batchlist

In [23]:
#count Batch: 6083 Complete!
batchlist = add_slopes(batchlist, 5)
batchlist = rolling_stats(batchlist, True, True)

Batch: 0 Complete!
Batch: 1 Complete!
Batch: 2 Complete!
Batch: 3 Complete!
Batch: 4 Complete!
Batch: 5 Complete!
Batch: 6 Complete!
Batch: 7 Complete!
Batch: 8 Complete!
Batch: 9 Complete!
Batch: 10 Complete!
Batch: 11 Complete!
Batch: 12 Complete!
Batch: 13 Complete!
Batch: 14 Complete!
Batch: 15 Complete!
Batch: 16 Complete!
Batch: 17 Complete!
Batch: 18 Complete!
Batch: 19 Complete!
Batch: 20 Complete!
Batch: 21 Complete!
Batch: 22 Complete!
Batch: 23 Complete!
Batch: 24 Complete!
Batch: 25 Complete!
Batch: 26 Complete!
Batch: 27 Complete!
Batch: 28 Complete!
Batch: 29 Complete!
Batch: 30 Complete!
Batch: 31 Complete!
Batch: 32 Complete!
Batch: 33 Complete!
Batch: 34 Complete!
Batch: 35 Complete!
Batch: 36 Complete!
Batch: 37 Complete!
Batch: 38 Complete!
Batch: 39 Complete!
Batch: 40 Complete!
Batch: 41 Complete!
Batch: 42 Complete!
Batch: 43 Complete!
Batch: 44 Complete!
Batch: 45 Complete!
Batch: 46 Complete!
Batch: 47 Complete!
Batch: 48 Complete!
Batch: 49 Complete!
Batch: 50 

Batch: 396 Complete!
Batch: 397 Complete!
Batch: 398 Complete!
Batch: 399 Complete!
Batch: 400 Complete!
Batch: 401 Complete!
Batch: 402 Complete!
Batch: 403 Complete!
Batch: 404 Complete!
Batch: 405 Complete!
Batch: 406 Complete!
Batch: 407 Complete!
Batch: 408 Complete!
Batch: 409 Complete!
Batch: 410 Complete!
Batch: 411 Complete!
Batch: 412 Complete!
Batch: 413 Complete!
Batch: 414 Complete!
Batch: 415 Complete!
Batch: 416 Complete!
Batch: 417 Complete!
Batch: 418 Complete!
Batch: 419 Complete!
Batch: 420 Complete!
Batch: 421 Complete!
Batch: 422 Complete!
Batch: 423 Complete!
Batch: 424 Complete!
Batch: 425 Complete!
Batch: 426 Complete!
Batch: 427 Complete!
Batch: 428 Complete!
Batch: 429 Complete!
Batch: 430 Complete!
Batch: 431 Complete!
Batch: 432 Complete!
Batch: 433 Complete!
Batch: 434 Complete!
Batch: 435 Complete!
Batch: 436 Complete!
Batch: 437 Complete!
Batch: 438 Complete!
Batch: 439 Complete!
Batch: 440 Complete!
Batch: 441 Complete!
Batch: 442 Complete!
Batch: 443 Co

Batch: 787 Complete!
Batch: 788 Complete!
Batch: 789 Complete!
Batch: 790 Complete!
Batch: 791 Complete!
Batch: 792 Complete!
Batch: 793 Complete!
Batch: 794 Complete!
Batch: 795 Complete!
Batch: 796 Complete!
Batch: 797 Complete!
Batch: 798 Complete!
Batch: 799 Complete!
Batch: 800 Complete!
Batch: 801 Complete!
Batch: 802 Complete!
Batch: 803 Complete!
Batch: 804 Complete!
Batch: 805 Complete!
Batch: 806 Complete!
Batch: 807 Complete!
Batch: 808 Complete!
Batch: 809 Complete!
Batch: 810 Complete!
Batch: 811 Complete!
Batch: 812 Complete!
Batch: 813 Complete!
Batch: 814 Complete!
Batch: 815 Complete!
Batch: 816 Complete!
Batch: 817 Complete!
Batch: 818 Complete!
Batch: 819 Complete!
Batch: 820 Complete!
Batch: 821 Complete!
Batch: 822 Complete!
Batch: 823 Complete!
Batch: 824 Complete!
Batch: 825 Complete!
Batch: 826 Complete!
Batch: 827 Complete!
Batch: 828 Complete!
Batch: 829 Complete!
Batch: 830 Complete!
Batch: 831 Complete!
Batch: 832 Complete!
Batch: 833 Complete!
Batch: 834 Co

Batch: 1170 Complete!
Batch: 1171 Complete!
Batch: 1172 Complete!
Batch: 1173 Complete!
Batch: 1174 Complete!
Batch: 1175 Complete!
Batch: 1176 Complete!
Batch: 1177 Complete!
Batch: 1178 Complete!
Batch: 1179 Complete!
Batch: 1180 Complete!
Batch: 1181 Complete!
Batch: 1182 Complete!
Batch: 1183 Complete!
Batch: 1184 Complete!
Batch: 1185 Complete!
Batch: 1186 Complete!
Batch: 1187 Complete!
Batch: 1188 Complete!
Batch: 1189 Complete!
Batch: 1190 Complete!
Batch: 1191 Complete!
Batch: 1192 Complete!
Batch: 1193 Complete!
Batch: 1194 Complete!
Batch: 1195 Complete!
Batch: 1196 Complete!
Batch: 1197 Complete!
Batch: 1198 Complete!
Batch: 1199 Complete!
Batch: 1200 Complete!
Batch: 1201 Complete!
Batch: 1202 Complete!
Batch: 1203 Complete!
Batch: 1204 Complete!
Batch: 1205 Complete!
Batch: 1206 Complete!
Batch: 1207 Complete!
Batch: 1208 Complete!
Batch: 1209 Complete!
Batch: 1210 Complete!
Batch: 1211 Complete!
Batch: 1212 Complete!
Batch: 1213 Complete!
Batch: 1214 Complete!
Batch: 121

Batch: 1543 Complete!
Batch: 1544 Complete!
Batch: 1545 Complete!
Batch: 1546 Complete!
Batch: 1547 Complete!
Batch: 1548 Complete!
Batch: 1549 Complete!
Batch: 1550 Complete!
Batch: 1551 Complete!
Batch: 1552 Complete!
Batch: 1553 Complete!
Batch: 1554 Complete!
Batch: 1555 Complete!
Batch: 1556 Complete!
Batch: 1557 Complete!
Batch: 1558 Complete!
Batch: 1559 Complete!
Batch: 1560 Complete!
Batch: 1561 Complete!
Batch: 1562 Complete!
Batch: 1563 Complete!
Batch: 1564 Complete!
Batch: 1565 Complete!
Batch: 1566 Complete!
Batch: 1567 Complete!
Batch: 1568 Complete!
Batch: 1569 Complete!
Batch: 1570 Complete!
Batch: 1571 Complete!
Batch: 1572 Complete!
Batch: 1573 Complete!
Batch: 1574 Complete!
Batch: 1575 Complete!
Batch: 1576 Complete!
Batch: 1577 Complete!
Batch: 1578 Complete!
Batch: 1579 Complete!
Batch: 1580 Complete!
Batch: 1581 Complete!
Batch: 1582 Complete!
Batch: 1583 Complete!
Batch: 1584 Complete!
Batch: 1585 Complete!
Batch: 1586 Complete!
Batch: 1587 Complete!
Batch: 158

...

Batch: 4515 Complete!
Batch: 4516 Complete!
Batch: 4517 Complete!
Batch: 4518 Complete!
Batch: 4519 Complete!
Batch: 4520 Complete!
Batch: 4521 Complete!
Batch: 4522 Complete!
Batch: 4523 Complete!
Batch: 4524 Complete!
Batch: 4525 Complete!
Batch: 4526 Complete!
Batch: 4527 Complete!
Batch: 4528 Complete!
Batch: 4529 Complete!
Batch: 4530 Complete!
Batch: 4531 Complete!
Batch: 4532 Complete!
Batch: 4533 Complete!
Batch: 4534 Complete!
Batch: 4535 Complete!
Batch: 4536 Complete!
Batch: 4537 Complete!
Batch: 4538 Complete!
Batch: 4539 Complete!
Batch: 4540 Complete!
Batch: 4541 Complete!
Batch: 4542 Complete!
Batch: 4543 Complete!
Batch: 4544 Complete!
Batch: 4545 Complete!
Batch: 4546 Complete!
Batch: 4547 Complete!
Batch: 4548 Complete!
Batch: 4549 Complete!
Batch: 4550 Complete!
Batch: 4551 Complete!
Batch: 4552 Complete!
Batch: 4553 Complete!
Batch: 4554 Complete!
Batch: 4555 Complete!
Batch: 4556 Complete!
Batch: 4557 Complete!
Batch: 4558 Complete!
Batch: 4559 Complete!
Batch: 456

Batch: 4895 Complete!
Batch: 4896 Complete!
Batch: 4897 Complete!
Batch: 4898 Complete!
Batch: 4899 Complete!
Batch: 4900 Complete!
Batch: 4901 Complete!
Batch: 4902 Complete!
Batch: 4903 Complete!
Batch: 4904 Complete!
Batch: 4905 Complete!
Batch: 4906 Complete!
Batch: 4907 Complete!
Batch: 4908 Complete!
Batch: 4909 Complete!
Batch: 4910 Complete!
Batch: 4911 Complete!
Batch: 4912 Complete!
Batch: 4913 Complete!
Batch: 4914 Complete!
Batch: 4915 Complete!
Batch: 4916 Complete!
Batch: 4917 Complete!
Batch: 4918 Complete!
Batch: 4919 Complete!
Batch: 4920 Complete!
Batch: 4921 Complete!
Batch: 4922 Complete!
Batch: 4923 Complete!
Batch: 4924 Complete!
Batch: 4925 Complete!
Batch: 4926 Complete!
Batch: 4927 Complete!
Batch: 4928 Complete!
Batch: 4929 Complete!
Batch: 4930 Complete!
Batch: 4931 Complete!
Batch: 4932 Complete!
Batch: 4933 Complete!
Batch: 4934 Complete!
Batch: 4935 Complete!
Batch: 4936 Complete!
Batch: 4937 Complete!
Batch: 4938 Complete!
Batch: 4939 Complete!
Batch: 494

Batch: 5271 Complete!
Batch: 5272 Complete!
Batch: 5273 Complete!
Batch: 5274 Complete!
Batch: 5275 Complete!
Batch: 5276 Complete!
Batch: 5277 Complete!
Batch: 5278 Complete!
Batch: 5279 Complete!
Batch: 5280 Complete!
Batch: 5281 Complete!
Batch: 5282 Complete!
Batch: 5283 Complete!
Batch: 5284 Complete!
Batch: 5285 Complete!
Batch: 5286 Complete!
Batch: 5287 Complete!
Batch: 5288 Complete!
Batch: 5289 Complete!
Batch: 5290 Complete!
Batch: 5291 Complete!
Batch: 5292 Complete!
Batch: 5293 Complete!
Batch: 5294 Complete!
Batch: 5295 Complete!
Batch: 5296 Complete!
Batch: 5297 Complete!
Batch: 5298 Complete!
Batch: 5299 Complete!
Batch: 5300 Complete!
Batch: 5301 Complete!
Batch: 5302 Complete!
Batch: 5303 Complete!
Batch: 5304 Complete!
Batch: 5305 Complete!
Batch: 5306 Complete!
Batch: 5307 Complete!
Batch: 5308 Complete!
Batch: 5309 Complete!
Batch: 5310 Complete!
Batch: 5311 Complete!
Batch: 5312 Complete!
Batch: 5313 Complete!
Batch: 5314 Complete!
Batch: 5315 Complete!
Batch: 531

Batch: 5644 Complete!
Batch: 5645 Complete!
Batch: 5646 Complete!
Batch: 5647 Complete!
Batch: 5648 Complete!
Batch: 5649 Complete!
Batch: 5650 Complete!
Batch: 5651 Complete!
Batch: 5652 Complete!
Batch: 5653 Complete!
Batch: 5654 Complete!
Batch: 5655 Complete!
Batch: 5656 Complete!
Batch: 5657 Complete!
Batch: 5658 Complete!
Batch: 5659 Complete!
Batch: 5660 Complete!
Batch: 5661 Complete!
Batch: 5662 Complete!
Batch: 5663 Complete!
Batch: 5664 Complete!
Batch: 5665 Complete!
Batch: 5666 Complete!
Batch: 5667 Complete!
Batch: 5668 Complete!
Batch: 5669 Complete!
Batch: 5670 Complete!
Batch: 5671 Complete!
Batch: 5672 Complete!
Batch: 5673 Complete!
Batch: 5674 Complete!
Batch: 5675 Complete!
Batch: 5676 Complete!
Batch: 5677 Complete!
Batch: 5678 Complete!
Batch: 5679 Complete!
Batch: 5680 Complete!
Batch: 5681 Complete!
Batch: 5682 Complete!
Batch: 5683 Complete!
Batch: 5684 Complete!
Batch: 5685 Complete!
Batch: 5686 Complete!
Batch: 5687 Complete!
Batch: 5688 Complete!
Batch: 568

Batch: 6018 Complete!
Batch: 6019 Complete!
Batch: 6020 Complete!
Batch: 6021 Complete!
Batch: 6022 Complete!
Batch: 6023 Complete!
Batch: 6024 Complete!
Batch: 6025 Complete!
Batch: 6026 Complete!
Batch: 6027 Complete!
Batch: 6028 Complete!
Batch: 6029 Complete!
Batch: 6030 Complete!
Batch: 6031 Complete!
Batch: 6032 Complete!
Batch: 6033 Complete!
Batch: 6034 Complete!
Batch: 6035 Complete!
Batch: 6036 Complete!
Batch: 6037 Complete!
Batch: 6038 Complete!
Batch: 6039 Complete!
Batch: 6040 Complete!
Batch: 6041 Complete!
Batch: 6042 Complete!
Batch: 6043 Complete!
Batch: 6044 Complete!
Batch: 6045 Complete!
Batch: 6046 Complete!
Batch: 6047 Complete!
Batch: 6048 Complete!
Batch: 6049 Complete!
Batch: 6050 Complete!
Batch: 6051 Complete!
Batch: 6052 Complete!
Batch: 6053 Complete!
Batch: 6054 Complete!
Batch: 6055 Complete!
Batch: 6056 Complete!
Batch: 6057 Complete!
Batch: 6058 Complete!
Batch: 6059 Complete!
Batch: 6060 Complete!
Batch: 6061 Complete!
Batch: 6062 Complete!
Batch: 606

### Creating a whole set


In [67]:
def createWholeset(batches, batchlimit, Min, Max):
    if batchlimit == True:
        batches = batches[Min:Max]
    
    startset = batches[0]
    count = 0
    
    for i in range(1, len(batches)):
        currentframe = batches[i]
        frames = [startset, currentframe]
        concatened_set = pd.concat(frames)
        
        startset = concatened_set
        print('Batch: ' + str(count) + ' concatenaded!!')
        count += 1

    return concatened_set

In [49]:
subds = createWholeset(batchlist, True, Min=0, Max=500)
subds1 = createWholeset(batchlist, True, Min=501, Max=1000)
subds2 = createWholeset(batchlist, True, Min=1001, Max=1500)
subds3 = createWholeset(batchlist, True, Min=1501, Max=2000)
subds4 = createWholeset(batchlist, True, Min=2001, Max=2500)
subds5 = createWholeset(batchlist, True, Min=2501, Max=3000)
subds6 = createWholeset(batchlist, True, Min=3501, Max=4000)
subds7 = createWholeset(batchlist, True, Min=4001, Max=4500)
subds8 = createWholeset(batchlist, True, Min=4501, Max=5000)
subds9 = createWholeset(batchlist, True, Min=5001, Max=5500)
subds10 = createWholeset(batchlist, True, Min=5501, Max=6000)
subds11 = createWholeset(batchlist, True, Min=6001, Max=6084)
subds12 = createWholeset(batchlist, True, Min=3001, Max=3500)

Batch: 0 concatenaded!!
Batch: 1 concatenaded!!
Batch: 2 concatenaded!!
Batch: 3 concatenaded!!
Batch: 4 concatenaded!!
Batch: 5 concatenaded!!
Batch: 6 concatenaded!!
Batch: 7 concatenaded!!
Batch: 8 concatenaded!!
Batch: 9 concatenaded!!
Batch: 10 concatenaded!!
Batch: 11 concatenaded!!
Batch: 12 concatenaded!!
Batch: 13 concatenaded!!
Batch: 14 concatenaded!!
Batch: 15 concatenaded!!
Batch: 16 concatenaded!!
Batch: 17 concatenaded!!
Batch: 18 concatenaded!!
Batch: 19 concatenaded!!
Batch: 20 concatenaded!!
Batch: 21 concatenaded!!
Batch: 22 concatenaded!!
Batch: 23 concatenaded!!
Batch: 24 concatenaded!!
Batch: 25 concatenaded!!
Batch: 26 concatenaded!!
Batch: 27 concatenaded!!
Batch: 28 concatenaded!!
Batch: 29 concatenaded!!
Batch: 30 concatenaded!!
Batch: 31 concatenaded!!
Batch: 32 concatenaded!!
Batch: 33 concatenaded!!
Batch: 34 concatenaded!!
Batch: 35 concatenaded!!
Batch: 36 concatenaded!!
Batch: 37 concatenaded!!
Batch: 38 concatenaded!!
Batch: 39 concatenaded!!
Batch: 40 

Batch: 320 concatenaded!!
Batch: 321 concatenaded!!
Batch: 322 concatenaded!!
Batch: 323 concatenaded!!
Batch: 324 concatenaded!!
Batch: 325 concatenaded!!
Batch: 326 concatenaded!!
Batch: 327 concatenaded!!
Batch: 328 concatenaded!!
Batch: 329 concatenaded!!
Batch: 330 concatenaded!!
Batch: 331 concatenaded!!
Batch: 332 concatenaded!!
Batch: 333 concatenaded!!
Batch: 334 concatenaded!!
Batch: 335 concatenaded!!
Batch: 336 concatenaded!!
Batch: 337 concatenaded!!
Batch: 338 concatenaded!!
Batch: 339 concatenaded!!
Batch: 340 concatenaded!!
Batch: 341 concatenaded!!
Batch: 342 concatenaded!!
Batch: 343 concatenaded!!
Batch: 344 concatenaded!!
Batch: 345 concatenaded!!
Batch: 346 concatenaded!!
Batch: 347 concatenaded!!
Batch: 348 concatenaded!!
Batch: 349 concatenaded!!
Batch: 350 concatenaded!!
Batch: 351 concatenaded!!
Batch: 352 concatenaded!!
Batch: 353 concatenaded!!
Batch: 354 concatenaded!!
Batch: 355 concatenaded!!
Batch: 356 concatenaded!!
Batch: 357 concatenaded!!
Batch: 358 c

Batch: 143 concatenaded!!
Batch: 144 concatenaded!!
Batch: 145 concatenaded!!
Batch: 146 concatenaded!!
Batch: 147 concatenaded!!
Batch: 148 concatenaded!!
Batch: 149 concatenaded!!
Batch: 150 concatenaded!!
Batch: 151 concatenaded!!
Batch: 152 concatenaded!!
Batch: 153 concatenaded!!
Batch: 154 concatenaded!!
Batch: 155 concatenaded!!
Batch: 156 concatenaded!!
Batch: 157 concatenaded!!
Batch: 158 concatenaded!!
Batch: 159 concatenaded!!
Batch: 160 concatenaded!!
Batch: 161 concatenaded!!
Batch: 162 concatenaded!!
Batch: 163 concatenaded!!
Batch: 164 concatenaded!!
Batch: 165 concatenaded!!
Batch: 166 concatenaded!!
Batch: 167 concatenaded!!
Batch: 168 concatenaded!!
Batch: 169 concatenaded!!
Batch: 170 concatenaded!!
Batch: 171 concatenaded!!
Batch: 172 concatenaded!!
Batch: 173 concatenaded!!
Batch: 174 concatenaded!!
Batch: 175 concatenaded!!
Batch: 176 concatenaded!!
Batch: 177 concatenaded!!
Batch: 178 concatenaded!!
Batch: 179 concatenaded!!
Batch: 180 concatenaded!!
Batch: 181 c

Batch: 459 concatenaded!!
Batch: 460 concatenaded!!
Batch: 461 concatenaded!!
Batch: 462 concatenaded!!
Batch: 463 concatenaded!!
Batch: 464 concatenaded!!
Batch: 465 concatenaded!!
Batch: 466 concatenaded!!
Batch: 467 concatenaded!!
Batch: 468 concatenaded!!
Batch: 469 concatenaded!!
Batch: 470 concatenaded!!
Batch: 471 concatenaded!!
Batch: 472 concatenaded!!
Batch: 473 concatenaded!!
Batch: 474 concatenaded!!
Batch: 475 concatenaded!!
Batch: 476 concatenaded!!
Batch: 477 concatenaded!!
Batch: 478 concatenaded!!
Batch: 479 concatenaded!!
Batch: 480 concatenaded!!
Batch: 481 concatenaded!!
Batch: 482 concatenaded!!
Batch: 483 concatenaded!!
Batch: 484 concatenaded!!
Batch: 485 concatenaded!!
Batch: 486 concatenaded!!
Batch: 487 concatenaded!!
Batch: 488 concatenaded!!
Batch: 489 concatenaded!!
Batch: 490 concatenaded!!
Batch: 491 concatenaded!!
Batch: 492 concatenaded!!
Batch: 493 concatenaded!!
Batch: 494 concatenaded!!
Batch: 495 concatenaded!!
Batch: 496 concatenaded!!
Batch: 497 c

Batch: 281 concatenaded!!
Batch: 282 concatenaded!!
Batch: 283 concatenaded!!
Batch: 284 concatenaded!!
Batch: 285 concatenaded!!
Batch: 286 concatenaded!!
Batch: 287 concatenaded!!
Batch: 288 concatenaded!!
Batch: 289 concatenaded!!
Batch: 290 concatenaded!!
Batch: 291 concatenaded!!
Batch: 292 concatenaded!!
Batch: 293 concatenaded!!
Batch: 294 concatenaded!!
Batch: 295 concatenaded!!
Batch: 296 concatenaded!!
Batch: 297 concatenaded!!
Batch: 298 concatenaded!!
Batch: 299 concatenaded!!
Batch: 300 concatenaded!!
Batch: 301 concatenaded!!
Batch: 302 concatenaded!!
Batch: 303 concatenaded!!
Batch: 304 concatenaded!!
Batch: 305 concatenaded!!
Batch: 306 concatenaded!!
Batch: 307 concatenaded!!
Batch: 308 concatenaded!!
Batch: 309 concatenaded!!
Batch: 310 concatenaded!!
Batch: 311 concatenaded!!
Batch: 312 concatenaded!!
Batch: 313 concatenaded!!
Batch: 314 concatenaded!!
Batch: 315 concatenaded!!
Batch: 316 concatenaded!!
Batch: 317 concatenaded!!
Batch: 318 concatenaded!!
Batch: 319 c

...

Batch: 202 concatenaded!!
Batch: 203 concatenaded!!
Batch: 204 concatenaded!!
Batch: 205 concatenaded!!
Batch: 206 concatenaded!!
Batch: 207 concatenaded!!
Batch: 208 concatenaded!!
Batch: 209 concatenaded!!
Batch: 210 concatenaded!!
Batch: 211 concatenaded!!
Batch: 212 concatenaded!!
Batch: 213 concatenaded!!
Batch: 214 concatenaded!!
Batch: 215 concatenaded!!
Batch: 216 concatenaded!!
Batch: 217 concatenaded!!
Batch: 218 concatenaded!!
Batch: 219 concatenaded!!
Batch: 220 concatenaded!!
Batch: 221 concatenaded!!
Batch: 222 concatenaded!!
Batch: 223 concatenaded!!
Batch: 224 concatenaded!!
Batch: 225 concatenaded!!
Batch: 226 concatenaded!!
Batch: 227 concatenaded!!
Batch: 228 concatenaded!!
Batch: 229 concatenaded!!
Batch: 230 concatenaded!!
Batch: 231 concatenaded!!
Batch: 232 concatenaded!!
Batch: 233 concatenaded!!
Batch: 234 concatenaded!!
Batch: 235 concatenaded!!
Batch: 236 concatenaded!!
Batch: 237 concatenaded!!
Batch: 238 concatenaded!!
Batch: 239 concatenaded!!
Batch: 240 c

Batch: 26 concatenaded!!
Batch: 27 concatenaded!!
Batch: 28 concatenaded!!
Batch: 29 concatenaded!!
Batch: 30 concatenaded!!
Batch: 31 concatenaded!!
Batch: 32 concatenaded!!
Batch: 33 concatenaded!!
Batch: 34 concatenaded!!
Batch: 35 concatenaded!!
Batch: 36 concatenaded!!
Batch: 37 concatenaded!!
Batch: 38 concatenaded!!
Batch: 39 concatenaded!!
Batch: 40 concatenaded!!
Batch: 41 concatenaded!!
Batch: 42 concatenaded!!
Batch: 43 concatenaded!!
Batch: 44 concatenaded!!
Batch: 45 concatenaded!!
Batch: 46 concatenaded!!
Batch: 47 concatenaded!!
Batch: 48 concatenaded!!
Batch: 49 concatenaded!!
Batch: 50 concatenaded!!
Batch: 51 concatenaded!!
Batch: 52 concatenaded!!
Batch: 53 concatenaded!!
Batch: 54 concatenaded!!
Batch: 55 concatenaded!!
Batch: 56 concatenaded!!
Batch: 57 concatenaded!!
Batch: 58 concatenaded!!
Batch: 59 concatenaded!!
Batch: 60 concatenaded!!
Batch: 61 concatenaded!!
Batch: 62 concatenaded!!
Batch: 63 concatenaded!!
Batch: 64 concatenaded!!
Batch: 65 concatenaded!!


Batch: 344 concatenaded!!
Batch: 345 concatenaded!!
Batch: 346 concatenaded!!
Batch: 347 concatenaded!!
Batch: 348 concatenaded!!
Batch: 349 concatenaded!!
Batch: 350 concatenaded!!
Batch: 351 concatenaded!!
Batch: 352 concatenaded!!
Batch: 353 concatenaded!!
Batch: 354 concatenaded!!
Batch: 355 concatenaded!!
Batch: 356 concatenaded!!
Batch: 357 concatenaded!!
Batch: 358 concatenaded!!
Batch: 359 concatenaded!!
Batch: 360 concatenaded!!
Batch: 361 concatenaded!!
Batch: 362 concatenaded!!
Batch: 363 concatenaded!!
Batch: 364 concatenaded!!
Batch: 365 concatenaded!!
Batch: 366 concatenaded!!
Batch: 367 concatenaded!!
Batch: 368 concatenaded!!
Batch: 369 concatenaded!!
Batch: 370 concatenaded!!
Batch: 371 concatenaded!!
Batch: 372 concatenaded!!
Batch: 373 concatenaded!!
Batch: 374 concatenaded!!
Batch: 375 concatenaded!!
Batch: 376 concatenaded!!
Batch: 377 concatenaded!!
Batch: 378 concatenaded!!
Batch: 379 concatenaded!!
Batch: 380 concatenaded!!
Batch: 381 concatenaded!!
Batch: 382 c

Batch: 166 concatenaded!!
Batch: 167 concatenaded!!
Batch: 168 concatenaded!!
Batch: 169 concatenaded!!
Batch: 170 concatenaded!!
Batch: 171 concatenaded!!
Batch: 172 concatenaded!!
Batch: 173 concatenaded!!
Batch: 174 concatenaded!!
Batch: 175 concatenaded!!
Batch: 176 concatenaded!!
Batch: 177 concatenaded!!
Batch: 178 concatenaded!!
Batch: 179 concatenaded!!
Batch: 180 concatenaded!!
Batch: 181 concatenaded!!
Batch: 182 concatenaded!!
Batch: 183 concatenaded!!
Batch: 184 concatenaded!!
Batch: 185 concatenaded!!
Batch: 186 concatenaded!!
Batch: 187 concatenaded!!
Batch: 188 concatenaded!!
Batch: 189 concatenaded!!
Batch: 190 concatenaded!!
Batch: 191 concatenaded!!
Batch: 192 concatenaded!!
Batch: 193 concatenaded!!
Batch: 194 concatenaded!!
Batch: 195 concatenaded!!
Batch: 196 concatenaded!!
Batch: 197 concatenaded!!
Batch: 198 concatenaded!!
Batch: 199 concatenaded!!
Batch: 200 concatenaded!!
Batch: 201 concatenaded!!
Batch: 202 concatenaded!!
Batch: 203 concatenaded!!
Batch: 204 c

Batch: 482 concatenaded!!
Batch: 483 concatenaded!!
Batch: 484 concatenaded!!
Batch: 485 concatenaded!!
Batch: 486 concatenaded!!
Batch: 487 concatenaded!!
Batch: 488 concatenaded!!
Batch: 489 concatenaded!!
Batch: 490 concatenaded!!
Batch: 491 concatenaded!!
Batch: 492 concatenaded!!
Batch: 493 concatenaded!!
Batch: 494 concatenaded!!
Batch: 495 concatenaded!!
Batch: 496 concatenaded!!
Batch: 497 concatenaded!!
Batch: 0 concatenaded!!
Batch: 1 concatenaded!!
Batch: 2 concatenaded!!
Batch: 3 concatenaded!!
Batch: 4 concatenaded!!
Batch: 5 concatenaded!!
Batch: 6 concatenaded!!
Batch: 7 concatenaded!!
Batch: 8 concatenaded!!
Batch: 9 concatenaded!!
Batch: 10 concatenaded!!
Batch: 11 concatenaded!!
Batch: 12 concatenaded!!
Batch: 13 concatenaded!!
Batch: 14 concatenaded!!
Batch: 15 concatenaded!!
Batch: 16 concatenaded!!
Batch: 17 concatenaded!!
Batch: 18 concatenaded!!
Batch: 19 concatenaded!!
Batch: 20 concatenaded!!
Batch: 21 concatenaded!!
Batch: 22 concatenaded!!
Batch: 23 concatena

In [106]:
len(concatened_set)+len(subds1)+len(subds2)+len(subds3)+len(subds4)+len(subds5)+len(subds6)+len(subds7)+len(subds8)+len(subds9)+len(subds10)+len(subds11)+len(subds12)

6084000

In [118]:
startset = concatened_set
currentframe = subds12
frames = [startset, currentframe]
concatened_set = pd.concat(frames)

print(len(concatened_set))

6084000


In [122]:
dataset = concatened_set
print(type(dataset))
print(dataset.shape)

<class 'pandas.core.frame.DataFrame'>
(6084000, 86)


### Save total merged file with extracted features
This file will be used for modelling part

In [ ]:
concatened_set.to_csv("Totalset+slopes+mean+sd.csv", index=False)